# 6. Deploy and Test Logic Apps
Now that we've verified that our Docker image works, this notebook will walk through how to operationalize this workflow using [Logic Apps](https://azure.microsoft.com/en-us/services/logic-apps/). 

For this workflow, adding a file whose name begins with 'trigger' will kick-off the entire batch style transfer process. In this architecture, we use Logic App as our trigger mechanism: when a trigger file (ex. 'trigger.txt') is uploaded into blob storage, Logic App will detect that the file has been added, and create an [Azure Container Instance (ACI)](https://azure.microsoft.com/en-us/services/container-instances/) to start creating Batch AI jobs.

This notebook will rely on the file __template.logic_app.json__ to deploy the Logic App. This file is a template for a [Azure Resource Manager (ARM)](https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-overview) json file, which we will use in this notebook to deploy the resources. In its current state, this file does not contain any of the parameters, such as those stored in the .env file, that we've set throughout the tutorial. To fill out those parameters, we will use [jinja](http://jinja.pocoo.org/) to populate the template with all the appropriate parameters and write the output to __logic_app.json__. This will be ARM json file that we use with the __az cli__ to deploy the Logic App.

As part of the Logic App deployment, we'll also deploy 2 additional resources: an Azure blob storage connector and an ACI connector. These connectors will allow Logic App to talk to these two resources.

---

Import utilities and load environment variables. In this notebook, we'll be using jinja2 to populate our Logic App deployment.

In [ ]:
import json
import os
import jinja2
%load_ext dotenv
%dotenv

## Deploy Logic App

Set variables we'll need for Logic App deployment:

- `logic_app` - the name of your logic app
- `aci_container_group` - the name of your ACI 
- `aci_display_name` - a display name used for your ACI deployment

In [ ]:
logic_app = "<my-logic-app>"
aci_container_group = "<my-aci-group>"
aci_display_name = "<your@email.com>"

Persist these variables to our `.env` file

In [ ]:
!dotenv set LOGIC_APP $logic_app
!dotenv set ACI_CONTAINER_GROUP $aci_container_group
!dotenv set ACI_DISPLAY_NAME $aci_display_name

Reload our environment variables from our `.env` file so we can use these variables as enviroment variables.

In [ ]:
%reload_ext dotenv
%dotenv

Using Jinja, populate the `template.logic_app.json` file and output the new file as `logic_app.json`. This file will be saved in the working directory. 

In [ ]:
# use jinja to fill in variables from .env file
env = jinja2.Environment(
    loader=jinja2.FileSystemLoader('.')
)
template = env.get_template('template.logic_app.json')

e = os.environ
rendered_template = template.render(env=e)

out = open('logic_app.json', 'w')
out.write(rendered_template)
out.close()

Now that we have the `logic_app.json` file, we can use the __az cli__ to deploy the resource.

In [ ]:
%%bash 
az group deployment create \
    --name $LOGIC_APP \
    --resource-group $RESOURCE_GROUP \
    --template-file logic_app.json

Once the Logic App is deployed, go into the Azure portal and open up the ACI connector and the Azure blob connector to authenticate. 

When you open up up the Azure ACI connector, it should look like this:

![azure_aci_connector_auth](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_aci_connector_auth.PNG)

When you open up up the Azure blob connector, it should look like this:

![azure_blob_connector_auth](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_blob_connector_auth.PNG)

For both of these connectors, click on the orange bar at the top to authenticate.

Once authenticated, your Logic App should be all set up and ready to trigger the workflow.

## Trigger Logic App
Logic App will be triggered whenever a blob that begins with the string 'trigger' is uploaded into the Azure blob storage account. Logic App will then read the contents of the file, using the contents as the directory of individual frames to apply style transfer to.

First we'll create a file titled `trigger_0.txt' and input the name of the content directory in blob to apply style transfer to.

In [ ]:
! touch trigger_0.txt && echo $FS_CONTENT_DIR > trigger_0.txt

Using AzCopy, upload the trigger file into blob to trigger Logic App.

In [ ]:
%%bash
azcopy \
    --source trigger_0.txt \
    --destination https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME}/trigger_0.txt \
    --dest-key $STORAGE_ACCOUNT_KEY

This file will trigger Logic App to create an ACI that will kick off the Batch AI jobs. We can now inspect the logs from ACI to see that everything is going smoothly:

In [ ]:
az container logs --resource-group $RESOURCE_GROUP --name $ACI_CONTAINER_GROUP

At this point, we can inspect the Azure portal to see all the moving parts:
- Logic Apps will be triggers and will spin up ACI
- ACI will break up the content images in blob and create BatchAI jobs
- The BatchAI cluster will scale up and start processing the work
- As the style transfer script is executed in batch on BatchAI, we will see the completed images (as well as logs) saved back to blob 

---

## Conclusion
In this notebook, we deployed Logic App and simulated what this end-to-end workflow could look like when operationalized with an add-to-blob trigger. At this point, I'd encourage you to take a look through the portal and examine the individual components of the architecture and make sure that all the parts are operating as expected. For further customization, you can also edit the Logic App via the Logic App Designer (in the portal) so that the triggering mechanism is more fitted to your scenarios.

The last step of the tutorial is to [download and re-stitch the individual frames so that we have a single style transferred video.](07_stitch_together_the_results.ipynb)